## NXOS设备配置
* feature bgp
* router bgp 65000
 * address-family ipv4 unicast

In [1]:
import xmltodict
from ncclient import manager

In [2]:
NXOS_netconf_info = {"host": "192.168.1.65",
                     "port": 830,
                     "username": "netconf",
                     "password": "netconf@123",
                     "hostkey_verify": False}

In [19]:
def nxconfig_bgp(method="get", config_filter=None, config_data=None):
    if method == "config":
        with manager.connect(**NXOS_netconf_info) as device:
            netconf_response = device.edit_config(target='running', config=config_data)
            return netconf_response
    else:
        with manager.connect(**NXOS_netconf_info) as device:
            netconf_response = device.get_config(source="running", filter=config_filter)
            bgp_data = xmltodict.parse(netconf_response.xml)
            return bgp_data['rpc-reply']['data']['System']['bgp-items']

## 完成BGP基础配置

In [20]:
bgp_baseline_config = '''<config>
    <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <bgp-items>
            <adminSt>enabled</adminSt>
            <inst-items>
                <adminSt>enabled</adminSt>
                <asn>65000</asn>
                <ctrl>fastExtFallover</ctrl>
                <disPolBatch>disabled</disPolBatch>
                <dom-items>
                    <Dom-list>
                        <rtrId>172.16.0.1</rtrId>
                        <name>default</name>
                        <af-items>
                            <DomAf-list>
                                <type>ipv4-ucast</type>
                                <advPip>disabled</advPip>
                                <advSysMac>disabled</advSysMac>
                                <advertL2vpnEvpn>disabled</advertL2vpnEvpn>
                                <allocLblAll>disabled</allocLblAll>
                                <clReflection>enabled</clReflection>
                                <critNhTimeout>3000</critNhTimeout>
                                <defInfOriginate>disabled</defInfOriginate>
                                <exportGwIp>disabled</exportGwIp>
                                <igpMetric>600</igpMetric>
                                <lblAllocMod>disabled</lblAllocMod>
                                <maxEcmp>1</maxEcmp>
                                <maxExtEcmp>1</maxExtEcmp>
                                <maxExtIntEcmp>1</maxExtIntEcmp>
                                <maxMxdEcmp>1</maxMxdEcmp>
                                <nonCritNhTimeout>10000</nonCritNhTimeout>
                                <prefix-items>
                                    <AdvPrefix-list>
                                        <addr>172.16.0.0/16</addr>
                                    </AdvPrefix-list>
                                </prefix-items>
                                <retainRttAll>disabled</retainRttAll>
                                <supprInactive>disabled</supprInactive>
                                <tblMapFltr>disabled</tblMapFltr>
                                <vniEthTag>disabled</vniEthTag>
                            </DomAf-list>
                        </af-items>
                        <always>disabled</always>
                        <bestPathIntvl>300</bestPathIntvl>
                        <holdIntvl>180</holdIntvl>
                        <kaIntvl>60</kaIntvl>
                        <mode>fabric</mode>
                        <peer-items>
                            <Peer-list>
                                <addr>172.16.0.2</addr>
                                <adminSt>enabled</adminSt>
                                <asn>65535</asn>
                                <bmpSrvId1St>disabled</bmpSrvId1St>
                                <bmpSrvId2St>disabled</bmpSrvId2St>
                                <capSuppr4ByteAsn>disabled</capSuppr4ByteAsn>
                                <epe>disabled</epe>
                                <holdIntvl>180</holdIntvl>
                                <inheritContPeerCtrl/>
                                <kaIntvl>60</kaIntvl>
                                <logNbrChgs>none</logNbrChgs>
                                <lowMemExempt>disabled</lowMemExempt>
                                <passwdType>LINE</passwdType>
                                <peerType>fabric-internal</peerType>
                                <privateASctrl>none</privateASctrl>
                            </Peer-list>
                        </peer-items>
                        <pfxPeerTimeout>30</pfxPeerTimeout>
                        <pfxPeerWaitTime>90</pfxPeerWaitTime>
                        <reConnIntvl>60</reConnIntvl>
                    </Dom-list>
                </dom-items>
                <evthist-items>
                    <EvtHist-list>
                        <type>cli</type>
                        <size>0</size>
                    </EvtHist-list>
                    <EvtHist-list>
                        <type>errors</type>
                        <size>0</size>
                    </EvtHist-list>
                    <EvtHist-list>
                        <type>events</type>
                        <size>0</size>
                    </EvtHist-list>
                </evthist-items>
                <flushRoutes>disabled</flushRoutes>
                <isolate>disabled</isolate>
                <name>bgp</name>
            </inst-items>
            <name>bgp</name>
        </bgp-items>
    </System>
</config>'''

In [21]:
netconf_response = nxconfig_bgp(method="config", config_data=bgp_baseline_config)

## 查看BGP基础配置

In [22]:
bgp_filter = """
<filter>
  <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <bgp-items/>
  </System>
</filter>
"""

In [23]:
bgp_data = nxconfig_bgp(method="get", config_filter=bgp_filter)

In [24]:
bgp_data

OrderedDict([('adminSt', 'enabled'),
             ('inst-items',
              OrderedDict([('adminSt', 'enabled'),
                           ('asn', '65000'),
                           ('ctrl', 'fastExtFallover'),
                           ('disPolBatch', 'disabled'),
                           ('dom-items',
                            OrderedDict([('Dom-list',
                                          OrderedDict([('name', 'default'),
                                                       ('af-items',
                                                        OrderedDict([('DomAf-list',
                                                                      OrderedDict([('type',
                                                                                    'ipv4-ucast'),
                                                                                   ('advPip',
                                                                                    'disabled'),
                      

## BGP Prefix

In [25]:
prefix = "10.99.99.0/24"

In [26]:
add_prefix = """ <config>
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <bgp-items>
        <inst-items>
            <dom-items>
                <Dom-list>
                    <name>default</name>
                    <af-items>
                        <DomAf-list>
                            <type>ipv4-ucast</type>
                            <prefix-items>
                                <AdvPrefix-list>
                                    <addr>{}</addr>
                                </AdvPrefix-list>
                            </prefix-items>
                        </DomAf-list>
                    </af-items>
                </Dom-list>
            </dom-items>
        </inst-items>
    </bgp-items>
</System>
</config>""".format(prefix)

In [27]:
nxconfig_bgp(method="config", config_data=add_prefix)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:039fedbb-bc70-4087-9a87-192d1fb720bc">
    <ok/>
</rpc-reply>

## 查看配置好的 Prefix

In [28]:
bgp_data = nxconfig_bgp(method="get", config_filter=bgp_filter)

In [29]:
bgp_data['inst-items']['dom-items']['Dom-list']['af-items']['DomAf-list']['prefix-items']['AdvPrefix-list']

[OrderedDict([('addr', '10.99.99.0/24')]),
 OrderedDict([('addr', '172.16.0.0/16')])]

## 删除配置好的 Prefix

In [32]:
del_prefix = """ <config>
<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <bgp-items>
        <inst-items>
            <dom-items>
                <Dom-list>
                    <name>default</name>
                    <af-items>
                        <DomAf-list>
                            <type>ipv4-ucast</type>
                            <prefix-items>
                                <AdvPrefix-list operation="remove">
                                    <addr>{}</addr>
                                </AdvPrefix-list>
                            </prefix-items>
                        </DomAf-list>
                    </af-items>
                </Dom-list>
            </dom-items>
        </inst-items>
    </bgp-items>
</System>
</config>""".format(prefix)

In [33]:
nxconfig_bgp(method="config", config_data=del_prefix)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7bd9d2cf-d83c-4548-8954-753ae69a4df1">
    <ok/>
</rpc-reply>

## 查看删除效果

In [34]:
bgp_data = nxconfig_bgp(method="get", config_filter=bgp_filter)
bgp_data['inst-items']['dom-items']['Dom-list']['af-items']['DomAf-list']['prefix-items']['AdvPrefix-list']

OrderedDict([('addr', '172.16.0.0/16')])